In [1]:
# Importing required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pickle


In [2]:
import os
os.getcwd()


'C:\\Users\\Aashish'

In [3]:
#2. Check the first few samples, shape, info of the data and try to familiarizeyourself with different features.
# Step 1: Read the dataset
data = pd.read_csv("loan_approval_data.csv")

print(data.head())

    loan_id gender married  dependents     education self_employed  \
0  lp001002   male      no         0.0      graduate            no   
1  lp001003   male     yes         1.0      graduate            no   
2  lp001005   male     yes         0.0      graduate           yes   
3  lp001006   male     yes         0.0  not graduate            no   
4  lp001008   male      no         0.0      graduate            no   

   applicantincome  coapplicantincome  loanamount  loan_amount_term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   credit_history property_area loan_status  
0             1.0         urban           y  
1             1.0         rural           n  
2             

In [4]:
print(data.shape)

(614, 13)


In [5]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   loan_id            614 non-null    object 
 1   gender             601 non-null    object 
 2   married            611 non-null    object 
 3   dependents         599 non-null    float64
 4   education          614 non-null    object 
 5   self_employed      582 non-null    object 
 6   applicantincome    614 non-null    int64  
 7   coapplicantincome  614 non-null    float64
 8   loanamount         592 non-null    float64
 9   loan_amount_term   600 non-null    float64
 10  credit_history     564 non-null    float64
 11  property_area      614 non-null    object 
 12  loan_status        614 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 62.5+ KB
None


In [6]:
#3. Check for missing values in the dataset, if present? handle them with appropriate methods and drop redundant features. (
# Check for missing values and handle them
print(data.isnull().sum())
data = data.dropna()


loan_id               0
gender               13
married               3
dependents           15
education             0
self_employed        32
applicantincome       0
coapplicantincome     0
loanamount           22
loan_amount_term     14
credit_history       50
property_area         0
loan_status           0
dtype: int64


In [7]:
if 'Loan_ID' in data.columns:
    data = data.drop(['Loan_ID'], axis=1)


In [2]:
#4. Visualize the distribution of the target column 'loan_status' with respect tovarious categorical features and write your observations. (2)
import seaborn as sns
import matplotlib.pyplot as plt

# Visualize loan_status distribution
sns.countplot(x='Loan_Status', data=data)
plt.show()


ValueError: Could not interpret value `Loan_Status` for `x`. An entry with this name does not appear in `data`.

In [3]:
#5. Encode the categorical data. (2)
from sklearn.preprocessing import LabelEncoder

# Encode categorical data
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data['Married'] = label_encoder.fit_transform(data['Married'])
data['Education'] = label_encoder.fit_transform(data['Education'])
data['Self_Employed'] = label_encoder.fit_transform(data['Self_Employed'])
data['Property_Area'] = label_encoder.fit_transform(data['Property_Area'])
data['Loan_Status'] = label_encoder.fit_transform(data['Loan_Status'])


KeyError: 'Gender'

In [4]:
#6. Separate the target and independent features and split the data into trainand test. (2)
from sklearn.model_selection import train_test_split

X = data.drop(columns=['Loan_Status'])
y = data['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


KeyError: "['Loan_Status'] not found in axis"

In [5]:
#7. Build any classification model to predict the loan status of the customerand save your model using pickle. (3)
from sklearn.ensemble import RandomForestClassifier
import pickle

# Build classification model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Save the model
with open('loan_eligibility_model.pkl', 'wb') as f:
    pickle.dump(clf, f)


NameError: name 'X_train' is not defined

In [ ]:
#2. app.py (30 points)
#● This is our main application file that is designed to call some of theimplemented APIs and methods, such as connecting to the MYSQLdatabase and creating a table for the database, and APIs such as userto register, login, enter_details, predict, and logout.
#a) Connect to the MYSQL database and create the class for the table.
#● User(id, username, password) (can create the table in the database using mysql workbench also)
#b) Register: This is a register API. This should take username and password
#and store the details in the user database.
#c) Login: This API should take the username and password of registered
#users and successfully log them in.
#d) Enter Details: This API should render the ‘predict.html’ page, where the
#user has to enter their details to check the loan eligibility.
#e) Predict: This API should render the prediction results on the HTML
#template.
#f) Logout: This API should logout the user.
from flask import Flask, render_template, request, redirect, session
from flask_mysqldb import MySQL
import pickle

app = Flask(__name__)

# Configure MySQL
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = 'password'  # Replace with your MySQL password
app.config['MYSQL_DB'] = 'loan_application'

mysql = MySQL(app)

# Load the pre-trained model
with open('loan_eligibility_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Home route
@app.route('/')
def home():
    return render_template('home.html')

# Register route
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        # Create cursor
        cur = mysql.connection.cursor()

        # Execute query
        cur.execute("INSERT INTO user(username, password) VALUES(%s, %s)", (username, password))

        # Commit to DB
        mysql.connection.commit()

        # Close connection
        cur.close()

        return redirect('/login')
    return render_template('register.html')

# Login route
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        # Create cursor
        cur = mysql.connection.cursor()

        # Execute query
        result = cur.execute("SELECT * FROM user WHERE username = %s AND password = %s", (username, password))

        if result > 0:
            session['logged_in'] = True
            session['username'] = username
            return redirect('/predict')
        else:
            return 'Invalid login'

        # Close connection
        cur.close()
    return render_template('login.html')

# Predict route
@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if 'logged_in' not in session:
        return redirect('/login')

    if request.method == 'POST':
        # Get user details from the form
        gender = request.form['gender']
        married = request.form['married']
        dependents = request.form['dependents']
        self_employed = request.form['self_employed']
        education = request.form['education']
        applicant_income = float(request.form['applicant_income'])
        coapplicant_income = float(request.form['coapplicant_income'])
        loan_amount = float(request.form['loan_amount'])
        loan_amount_term = float(request.form['loan_amount_term'])
        credit_history = request.form['credit_history']
        property_area = request.form['property_area']

        # Prepare input for prediction
        input_data = [[gender, married, dependents, self_employed, education, applicant_income, coapplicant_income, 
                       loan_amount, loan_amount_term, credit_history, property_area]]
        
        # Make prediction using the pre-trained model
        prediction = model.predict(input_data)

        # Render prediction results on the HTML template
        return render_template('prediction.html', prediction=prediction[0])
        
    return render_template('predict.html')

# Logout route
@app.route('/logout')
def logout():
    session.clear()
    return redirect('/login')

if __name__ == '__main__':
    app.secret_key = 'secret123'  # Set a secret key for session management
    app.run(debug=True)
